# Adleman's algorithm for discrete logarithms


This program was made for an assignment of the class ”Cryptography” of my master’s program:  
"Use the Adleman algorithm to calculate the discrete logarithm $x$ inside $\mathbb{Z}_N=\mathbb{Z}_{1693}$, where $17^x \equiv 101 \, mod \, 1693$."

<u>**Adleman's algorithm**</u>  
$\cdot$ Input: A prime $N$, a primitive root $g \, mod \, N$ with $g \in \{2,\dots,p-2\}$ and $a \in \{1,\dots,p-1\}$  
$\cdot$ Output: $x=\log_g a$
1. We choose $B \in \mathbb{Z}^{+}$ with $B<N$ and define the set $F(B)$ of all primes $\leq B$.
2. For every $q \in F(B)$, we compute* the discrete logarithm $x(q)=\log_g q$ inside $\mathbb{Z}_{N}^*$.
3. We determine an integer $y \in \{0,\dots,N-1\}$ with $ag^y \equiv \prod_{q \in F(B)} q^{e(q)} \, (mod \, N)$, where $e(q)$ is an integer, $\geq 0$.
4. The requested discrete logarithm can be computed by $\log_g a = \sum_{q \in F(B)} x(q)e(q)-y \, [mod \, (N-1)]$.

*In our implementation of the algorithm, the discrete logarithms will be computed using Pollard's algorithm for discrete logarithms. In theory, another way is proposed in order to compute them, but it won't be necessary here.

Author: Florias Papadopoulos

## Importing modules

We start by importing the modules that we will use

In [1]:
import math

## Defining the functions

### Starter functions

#### (a) F

We will need a function that can compute the set $F(B)$, given an integer $B$.  
For example, if $B=10$, then $F(B)=\{2,3,5,7\}$.

In [2]:
def F(B):
    base = []
    for i in range(2, B + 1):
        for j in range(2, int(i ** 0.5) + 1):
            if i%j == 0:
                break
        else:
            base.append(i)
    return base

#### (b) multiplyList

We will also need a function that can multiply all elements in a given "python list".

In [3]:
def multiplyList(xList) :
    
    product = 1
    for x in xList:
         product = product * x
            
    return product

### Pollard's algorithm functions

We will use the Pollard algorithm in order to compute the discrete logarithms on the 2nd step of the algorithm.  
To do so, we import all functions related to the Pollard algorithm. More information on them can be found on the accompagnying .ipynb file.

*We note that $n=|<g>|$.

In [4]:
def func(N,n,g,beta,x,a,b,S_1,S_2,S_3):

  #upper case
  if x in S_1:
        one = (beta*x) % N
        two = a % n
        three = (b+1) % n
        return one, two, three

  #middle case
  if x in S_2:
        one = (x**2) % N
        two = (2*a) % n
        three = (2*b) % n
        return one, two, three

  #bottom case
  if x in S_3:
        one = (g*x) % N
        two = (a+1) % n
        three = b % n
        return one, two, three
    
#---------------------------------------------------------------
#---------------------------------------------------------------
def tri_seq(N,n,g,beta,S_1,S_2,S_3,i_upper):

    tri_seq_list = [[1,0,0]]
    x_i, a_i, b_i = 1, 0, 0
    for i in range(0,i_upper+1):
        tri_seq_sub_list = []
        (x_next, a_next, b_next) = func(N,n,g,beta,x_i,a_i,b_i,S_1,S_2,S_3)
        tri_seq_sub_list.append(x_next)
        tri_seq_sub_list.append(a_next)
        tri_seq_sub_list.append(b_next)
        tri_seq_list.append(tri_seq_sub_list)
        x_i = x_next
        a_i = a_next
        b_i = b_next
    return tri_seq_list

#---------------------------------------------------------------
#---------------------------------------------------------------
def modulo_solver(b,a,n):
    
    #print(b,"* x =",a,"(modn)")
    #print("=>")
    
    x_list = []
    if b != 0 and b == a:
        x_list.append(1)
    if b != 0 and b !=a :
        gcd = math.gcd(math.gcd(b,a),n)
        b_gcd = b//gcd
        a_gcd = a//gcd
        n_gcd = n//gcd
        
        #print(b_gcd,"* x =",a_gcd,"(",n_gcd,")")
        if b_gcd == 1:
            x = a_gcd % n_gcd
        if b_gcd == -1:
            x = (-1)*a_gcd % n_gcd
        if b_gcd != 1 and b_gcd !=-1:
            b_inv = pow(b_gcd, -1,n_gcd)
            x = (b_inv*a_gcd) % n_gcd
    
        for k in range(0,gcd):
            x_k = k*n_gcd + x
            x_list.append(x_k)
            
    return x_list

#---------------------------------------------------------------
#---------------------------------------------------------------
def verify(N,g,beta,solution):
    return pow(g,solution,N) == beta

#---------------------------------------------------------------
#---------------------------------------------------------------
def discr_pollardAlgorithm(N,n,g,beta,i_upper):
    
    #error message in case i_upper is not enough
    proper_solution = "problem, increase i_upper"

    #creating S_1,S_2,S_3
    S_1 = set()
    S_2 = set()
    S_3 = set()

    for x in range(1,N):
        if (x-1) % 3 == 0:
            S_1.add(x)
        if x % 3 == 0:
            S_2.add(x)
        if (x-2) % 3 == 0:
            S_3.add(x)

    #creating tri_seq
    seq = tri_seq(N,n,g,beta,S_1,S_2,S_3,i_upper)

    #finding the solutions z
    z_list = []

    for t in range(1,math.floor(i_upper/2)):
        [x_t, a_t, b_t] = seq[t]
        [x_2t, a_2t, b_2t] = seq[2*t]

        if x_t == x_2t:
            b_minus = b_2t - b_t
            a_minus = a_t - a_2t
            z_list = modulo_solver(b_minus,a_minus,n)
            
            #finding and verifying the correct z from the list that contains all of the solutions z
            for z in z_list:
                if verify(N,g,beta,z) == True:
                    proper_solution = z
                    break
            break

    return S_1, S_2, S_3, seq, t, x_t, x_2t, b_t, b_2t, a_t, a_2t, z_list, proper_solution

### Adleman function

We will now create our main function based on the Adleman algorithm.  
This will be the only problem that we will be basically solving partially, only for the case $B=11$, which is the one that we are interested in for our problem.  
Moreover, we will use as input again an index $i_{upper}$ for the needs of the Pollard algorithm for discrete logarithms, as well as another index $e_{upper}$ which will be used in order to control the size of the "multi_list", which is basically a set $\{ \prod_{q \in F(B)} q^{e(q)} \, (mod \, N) \, | \, 0 \leq e(q) \leq e_{upper} \}$.

In [5]:
def adlemanAlgorithm(N,n,g,a,i_upper,e_upper):
    
    B=11 #B=11 is perfect for our problem
        
    #computing the set F(B)
    F_B = F(B)
    
    #finding the discretes logarithms of step2
    X_q = []
    for q in F_B:
        (S_1, S_2, S_3, seq, t, x_t, x_2t, b_t, b_2t, a_t, a_2t, z_list, proper_solution) = discr_pollardAlgorithm(N,n,g,q,i_upper)
        X_q.append(proper_solution)
        
    #calculating the product of the modular equation for F(B)=F(11)={1,2,3,5,7,11}
    all_list = []
    multi_list = []
    for e_0 in range(e_upper+1):
        for e_1 in range(e_upper+1):
            for e_2 in range(e_upper+1):
                for e_3 in range(e_upper+1):
                    for e_4 in range(e_upper+1):
                        multi = (((F_B[0] ** e_0) % N) * ((F_B[1] ** e_1) % N) * ((F_B[2] ** e_2) % N) * \
                                 ((F_B[3] ** e_3) % N) * ((F_B[4] ** e_4) % N)) % N
                        all_list.append((multi, e_0, e_1, e_2, e_3, e_4))
                        multi_list.append(multi)

    #checking if there is a y such that agy=ag^y belongs in the "multi_list"
    discr_log = 0
    for y in range(N):
        agy = (a*(g**y)) % N
        
        #when such a y is found, the discrete logarithm is computed as mentioned in step4
        if agy in multi_list:
            index_agy = multi_list.index(agy)
            (multi, e_0, e_1, e_2, e_3, e_4) = all_list[index_agy]
            discr_log = (X_q[0]*e_0 + X_q[1]*e_1 + X_q[2]*e_2 + X_q[3]*e_3 + X_q[4]*e_4 -y) % n #n=N-1
            break

    return F_B, X_q, y, multi, e_0, e_1, e_2, e_3, e_4, discr_log


## Solving the problem

We create a script that uses the above function to return us a text that elaborates on all the values that were computed in each step of the algorithm.  
In our problem $N=1693$, $n=1692$, $g=17$ and $a =101$.

In [6]:
#input
N, n, g, a = 1693, 1692, 17, 101
i_upper, e_upper = 1000, 4
#input

(F_B, X_q, y, multi, e_0, e_1, e_2, e_3, e_4, discr_log) = adlemanAlgorithm(N,n,g,a,i_upper,e_upper)

print("We wanted to find the discrete logarithm log_" +str(g) + "(" + str(a)+ ")")
print("For this, we used Adleman's Algorithm and did the following:")
print("")
print("In step 1, we chose B=11 and formed the set F(B)=", F_B)
print("In step 2, for each q in F(B) we calculated the discrete logarithm X(q), getting the set", X_q) 
print("In step 3, we found y= " + str(y) + " such that ag^y (equiv) Π_(q in F(B))[q^e(q)] (equiv) " + str(multi) + " (modp), with e_0=" +str(e_0) + ", e_1=" +str(e_1) + ", e_2=" + str(e_2) + ", e_3=" + str(e_3) + ", e_4=" + str(e_4))
print("In step 4, we calculated the discrete logarithm log_" +str(g) + "(" + str(a)+ ")=" + str(discr_log) + " that we were looking for calculating the last sum of the algorithm")

We wanted to find the discrete logarithm log_17(101)
For this, we used Adleman's Algorithm and did the following:

In step 1, we chose B=11 and formed the set F(B)= [2, 3, 5, 7, 11]
In step 2, for each q in F(B) we calculated the discrete logarithm X(q), getting the set [1201, 842, 187, 1085, 549]
In step 3, we found y= 1 such that ag^y (equiv) Π_(q in F(B))[q^e(q)] (equiv) 24 (modp), with e_0=3, e_1=1, e_2=0, e_3=0, e_4=0
In step 4, we calculated the discrete logarithm log_17(101)=1060 that we were looking for calculating the last sum of the algorithm
